For more details see https://skeptric.com/sparql-analysing-rdf

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import gzip
import rdflib
import urllib
from urllib.request import urlretrieve
from pathlib import Path

import dateutil
import datetime

import logging

from tqdm.notebook import tqdm

In [3]:
sys.path.insert(0, '../src')

In [4]:
from lib.rdftool import *

Data From http://webdatacommons.org/structureddata/2019-12/stats/schema_org_subsets.html

Download both the microdata (1.9GB) and the JSON-LD (700MB)

In [5]:
DEST_DIR = Path('..') / 'data' / 'webcommons'
DEST_DIR.mkdir(parents=True, exist_ok=True)

In [6]:
class TqdmUpTo(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)  # will also set self.n = b * bsize

def download(url, filename, overwrite=False):
    filename = Path(filename)
    if (not filename.exists()) or overwrite:
        with TqdmUpTo(unit = 'B', unit_scale = True, unit_divisor = 1024, miniters = 1, desc = Path(filename).name) as t:
            urlretrieve(url, filename = filename, reporthook = t.update_to)

In [7]:
JOBS_JSON_2019 = DEST_DIR / '2019-12_json_JobPosting.gz'

In [8]:
JOBS_MD_2019 = DEST_DIR / '2019-12_md_JobPosting.gz'

In [9]:
download('http://data.dws.informatik.uni-mannheim.de/structureddata/2019-12/quads/classspecific/json/schema_JobPosting.gz',
         JOBS_JSON_2019)

In [10]:
download('http://data.dws.informatik.uni-mannheim.de/structureddata/2019-12/quads/classspecific/md/schema_JobPosting.gz',
         JOBS_MD_2019)

# JSON

In [11]:
def get_domain(url):
    return urllib.parse.urlparse(url).netloc

In [12]:
def parse_nquads_distinct(lines, total=None):
    seen = set()
    for group, quad_lines in tqdm(groupby(lines, get_quad_label), total=total):
        domain = get_domain(group)
        if domain in seen:
            continue
        graph = rdflib.Graph(identifier=group)
        try:
            graph.parse(data=''.join(quad_lines), format='nquads')
        except rdflib.plugins.parsers.ntriples.ParseError as e:
            msg = str(e)
            if len(msg) > 255:
                msg = msg[:255] + '...'
            logging.error(str(msg))
            continue
        if list(get_job_postings(graph)):
            seen.add(domain)
            yield graph

In [13]:
rdflib.term.bind(rdflib.term.URIRef("http://schema.org/Date"), datetime.datetime, dateutil.parser.parse, lambda dt: dt.isoformat())
rdflib.term.bind(rdflib.term.URIRef("http://schema.org/DateTime"), datetime.datetime, dateutil.parser.parse, lambda dt: dt.isoformat())
rdflib.term.bind(rdflib.term.URIRef("https://schema.org/Date"), datetime.datetime, dateutil.parser.parse, lambda dt: dt.isoformat())
rdflib.term.bind(rdflib.term.URIRef("https://schema.org/DateTime"), datetime.datetime, dateutil.parser.parse, lambda dt: dt.isoformat())

In [14]:
json_graphs = list(parse_nquads_distinct(gzip.open(JOBS_JSON_2019, 'rt'), 460_000))

/home/eross/.virtualenvs/ds/lib/python3.6/site-packages/dateutil/parser/_parser.py:1204: UnknownTimezoneWarning: tzname CET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
skype:raloffice?call|skype:raloffice?chat does not look like a valid URI, trying to serialize this will break.
https://www.accenture.com/us-en?c=us_us_brand_10460943&amp;n=psgs_brand_1218&amp;c=ad_usadfy17_10000001&amp;n=psgs_Brand-|-US-|-Exact_accenture&amp;gclid=EAIaIQobChMIpKXKyq2o5AIVksDACh36_QtlEAAYASAAEgLCbPD_BwE does not look like a valid URI, trying to serialize this will break.


In [15]:
md_graphs = list(parse_nquads_distinct(gzip.open(JOBS_MD_2019, 'rt'), 680_000))

http://chart.apis.google.com/chart?chs=155x155&cht=qr&chl=http%3A%2F%2Fwww%2Eemprega%2Einfo%2FVA%2D93231%2Demprego%2Dde%2DScrum%2DMaster%2Dem%2DCampinas%2DSP&chld=|0 does not look like a valid URI, trying to serialize this will break.
https://www.flexibleworks.co.uk/homepage.php?employerid=16&company=Citizen\ does not look like a valid URI, trying to serialize this will break.
https://www.diversitylink.co.uk/homepage.php?employerid=122&company=St-Andrew\ does not look like a valid URI, trying to serialize this will break.
https://cbb.de/karriere/stellenboerse/detail/s-51-h-entwicklungsingenieur-entwicklungsingenieur-projektleiter-dipl-ing-elektrotechnik-dipl/javascript:linkTo_UnCryptMailto('nbjmup+kpctAdcc\/ef'); does not look like a valid URI, trying to serialize this will break.
ERROR:root:Invalid line (Failed to eat <([^:]+:[^\s"<>]*)> at <http://schema.org/JobPosting/\"description\"> "\n\t\t\t\t\u0420\u0430\u0431\u043E\u0442\u0430 \u043D\u0430 \u0437\u0430\u0432\u043E\u0434\u0435 \

ERROR:root:Invalid line (Failed to eat <([^:]+:[^\s"<>]*)> at <http://schema.org/JobPosting/\"description\"> "\n\t\t\t\t\u0420\u0410\u0411\u041E\u0422\u0410 \u0412 \u0427\u0415\u0425\u0418\u0418 \u041D\u0410 \u0428\u041E\u041A\u041E\u041B\u0410\u0414\u041D\u041E\u04...
ERROR:root:Invalid line (Failed to eat <([^:]+:[^\s"<>]*)> at <http://schema.org/JobPosting/\"description\"> "\n\t\t\t\t\u0421\u043E\u0441\u0442\u0430\u0432\u043B\u0435\u043D\u0438\u044F \u0438 \u043A\u043E\u043C\u043F\u043B\u0435\u043A\u0442\u0430\u0446\u0438\u044F...
ERROR:root:Invalid line (Failed to eat <([^:]+:[^\s"<>]*)> at <http://schema.org/JobPosting/\"description\"> "\n\t\t\t\t\u0420\u0430\u0431\u043E\u0442\u0430 \u0432 \u0427\u0435\u0445\u0438\u0438 \u0422\u0440\u0430\u043A\u0442\u043E\u0440\u0438\u0441\u0442\u043E\u043...
ERROR:root:Invalid line (Failed to eat <([^:]+:[^\s"<>]*)> at <http://schema.org/JobPosting/\"description\"> "\n\t\t\t\t\u0418\u0437\u0433\u043E\u0442\u043E\u0432\u043B\u0435\u043D\u0438\u04

In [16]:
len(json_graphs)

5376

In [17]:
len(md_graphs)

10087

In [18]:
dataset = rdflib.Dataset()
for graph in json_graphs + md_graphs:
    dataset.add_graph(graph)

# Starting SPARQL

In [19]:
results = dataset.query('''
SELECT ?src
WHERE {
GRAPH ?src
{}
}
limit 5
''')

for row in results:
    print(row)

(rdflib.term.URIRef('https://avantripbiblos.catsone.com/careers/3067-General/jobs/11459279-UX-Designer/'),)
(rdflib.term.URIRef('https://pushkino.rabota.ru/vacancy/%D0%B3%D1%80%D1%83%D0%B7%D1%87%D0%B8%D0%BA-%D1%80%D0%B0%D0%B7%D0%BD%D0%BE%D1%80%D0%B0%D0%B1%D0%BE%D1%87%D0%B8%D0%B9/70000%20%D1%80%D1%83%D0%B1/'),)
(rdflib.term.URIRef('https://www.ajilon.com/jobs/payment-poster/?ID=US_EN_7_843316_2709099'),)
(rdflib.term.URIRef('https://www.barnardmarcus.co.uk/about-us'),)
(rdflib.term.URIRef('https://it.careercast.com/jobs/product-analyst-data-scientist-gopro-san-diego-california-115313678-d?contextType=browse'),)


In [20]:
def head(items, n=5):
    i = 0
    for item in items:
        i += 1
        if i <= n:
            print([field.toPython() if field else field for field in item])
        # Exhaust the iterator, otherwise we get strange behaviour

15,458 Jobs

In [21]:
results = dataset.query('''
SELECT (COUNT(?src) as ?count)
WHERE {
GRAPH ?src
{}
}
''')

head(results)

[15458]


38,760 job postings

In [22]:
results = dataset.query('''
PREFIX sdo: <http://schema.org/>
SELECT (COUNT(?src) as ?count)
WHERE {
GRAPH ?src
{[] a sdo:JobPosting . }
}
''')

head(results)

[38760]


Only 13k of the 15k graphs have exactly one job posting

In [23]:
results = dataset.query('''
PREFIX sdo: <http://schema.org/>
SELECT (COUNT(?src) as ?count) {
SELECT ?src 
WHERE {
GRAPH ?src
{[] a sdo:JobPosting . }
}
GROUP BY ?src
HAVING (COUNT(?src) = 1)
}
''')

head(results)

[13112]


# Check the schema

In [24]:
%%time
results = dataset.query('''
SELECT ?schema (COUNT(?src) AS ?total)
WHERE {
GRAPH ?src
{
{_:j a <http://schema.org/JobPosting> ;
     a ?schema .
}
UNION
{_:j a <https://schema.org/JobPosting> ;
     a ?schema .
}
}
}
GROUP BY ?schema
LIMIT 5
''')

head(results)

['http://schema.org/JobPosting', 38760]
['https://schema.org/JobPosting', 21]
CPU times: user 4.19 s, sys: 219 ms, total: 4.41 s
Wall time: 4.53 s


In [25]:
%%time
results = dataset.query('''
SELECT ?type (COUNT(?type) as ?total)
WHERE {
    GRAPH ?src
    {_:j a ?type . FILTER (REGEX(str(?type), 'https?://schema.org/JobPosting'))}
}
GROUP BY ?type
ORDER BY DESC(?total)
''')


head(results)

['http://schema.org/JobPosting', 38760]
['https://schema.org/JobPosting', 21]
CPU times: user 15 s, sys: 297 ms, total: 15.3 s
Wall time: 15.7 s


In [26]:
%%time
results = dataset.query('''
SELECT ?type (count(?src) as ?count)
WHERE {
    GRAPH ?src
    {_:j a ?type .}
    VALUES ?type { <http://schema.org/JobPosting> <https://schema.org/JobPosting> }
}
GROUP BY ?type
ORDER BY DESC(?count)
LIMIT 5
''')


head(results)

['http://schema.org/JobPosting', 38760]
['https://schema.org/JobPosting', 21]
CPU times: user 14.9 s, sys: 719 ms, total: 15.6 s
Wall time: 16.1 s


# Just getting the singles

Having lots of job ads from a single source could lead to unrealistic results

In [27]:
%%time
results = dataset.query('''
SELECT ?schema (COUNT(?src) AS ?total)
WHERE {
GRAPH ?src
{
{_:j a <http://schema.org/JobPosting> ;
     a ?schema .
}
UNION
{_:j a <https://schema.org/JobPosting> ;
     a ?schema .
}
}
}
GROUP BY ?schema
LIMIT 5
''')

head(results)

['http://schema.org/JobPosting', 38760]
['https://schema.org/JobPosting', 21]
CPU times: user 4.36 s, sys: 141 ms, total: 4.5 s
Wall time: 4.65 s


In [28]:
%%time
results = dataset.query('''
SELECT ?schema ?src
WHERE {
GRAPH ?src
{_:j a <https://schema.org/JobPosting> ;
     a ?schema .
}
}
''')

head(results)

['https://schema.org/JobPosting', 'https://trabaja.medicosdelmundo.org/empleos/oferta/408518/Psychologue-Republique-democratique-du-Congo.html']
['https://schema.org/JobPosting', 'https://www.trabajando.com.co/empleos/oferta/836292/Foreman---Team-leader-Stripping-Area-E-Scrap-business.html']
['https://schema.org/JobPosting', 'https://www.trabajando.es/empleos/oferta/412531/OPERARIO-A-CON-FUCNION-DE-LIMPIEZA.html']
['https://schema.org/JobPosting', 'https://exalumnosudec.trabajando.com/empleos/ofertas/3782882/Ingeniero-Estudios-de-Mercado.html']
['https://schema.org/JobPosting', 'https://bolsalaboralucn.trabajando.com/empleos/ofertas/3841806/Tecnico-informatico.html']
CPU times: user 688 ms, sys: 78.1 ms, total: 766 ms
Wall time: 810 ms


In [29]:
multiple_job_graphs = dataset.query('''
PREFIX sdo: <http://schema.org/>
PREFIX sdos: <https://schema.org/>
SELECT ?src 
WHERE {
GRAPH ?src
{
{[] a sdo:JobPosting . }
UNION
{[] a sdos:JobPosting .}
}
}
GROUP BY ?src
HAVING (COUNT(?src) > 1)
''')

multiple_job_graphs = set(x[0] for x in multiple_job_graphs)

In [30]:
len(multiple_job_graphs)

2366

In [31]:
remove_graphs = []
for graph in dataset.graphs():
    if graph.identifier in multiple_job_graphs:
        remove_graphs.append(graph)
    
for graph in remove_graphs:
    dataset.remove_graph(graph)

In [32]:
for idx, _graph in enumerate(dataset.graphs()):
    pass
idx + 1

13093

# Types

In [33]:
%%time
results = dataset.query('''
SELECT ?type (COUNT(?src) AS ?postings) (SUM(?n) as ?total) {
SELECT ?src ?type (COUNT(?type) AS ?n)
WHERE {
    GRAPH ?src
    {_:j a ?type .}
}
GROUP BY ?src ?type
}
GROUP BY ?type
HAVING (COUNT(?src) > 50)
ORDER BY desc(?total)
''')


head(results, 20)

['http://schema.org/JobPosting', 13092, 13092]
['http://schema.org/Place', 8734, 9301]
['http://schema.org/Organization', 7972, 9184]
['http://schema.org/PostalAddress', 8065, 9018]
['http://schema.org/MonetaryAmount', 2958, 2970]
['http://schema.org/PropertyValue', 2875, 2882]
['http://schema.org/ListItem', 946, 2871]
['http://schema.org/QuantitativeValue', 2602, 2619]
['http://schema.org/ImageObject', 609, 1057]
['http://schema.org/BreadcrumbList', 939, 988]
['http://schema.org/WebSite', 770, 790]
['http://schema.org/WebPage', 493, 672]
['http://schema.org/LocalBusiness', 420, 557]
['http://schema.org/ItemPage', 501, 501]
['http://schema.org/GeoCoordinates', 438, 493]
['http://schema.org/SearchAction', 363, 371]
['http://schema.org/SiteNavigationElement', 221, 353]
['http://data-vocabulary.org/Organization', 347, 348]
['http://schema.org/Country', 343, 344]
['http://data-vocabulary.org/Breadcrumb', 150, 342]
CPU times: user 25.3 s, sys: 500 ms, total: 25.8 s
Wall time: 27 s


In [34]:
import pandas as pd

In [35]:
df = pd.DataFrame([[value.toPython() for value in row] for row in results], columns = ['uri', 'n', 'total'])

In [36]:
df.assign(frac=lambda df: df.n/max(df.n)).assign(avg = lambda df: df.total / df.n).head(15)

,uri,n,total,frac,avg
0,http://schema.org/JobPosting,13092,13092,1.000000,1.000000
1,http://schema.org/Place,8734,9301,0.667125,1.064919
2,http://schema.org/Organization,7972,9184,0.608921,1.152032
3,http://schema.org/PostalAddress,8065,9018,0.616025,1.118165
4,http://schema.org/MonetaryAmount,2958,2970,0.225940,1.004057
5,http://schema.org/PropertyValue,2875,2882,0.219600,1.002435
6,http://schema.org/ListItem,946,2871,0.072258,3.034884
7,http://schema.org/QuantitativeValue,2602,2619,0.198747,1.006533
8,http://schema.org/ImageObject,609,1057,0.046517,1.735632
9,http://schema.org/BreadcrumbList,939,988,0.071723,1.052183


## Properties of Job

In [46]:
%%time
results = dataset.query('''
PREFIX sdo: <http://schema.org/>
SELECT ?rel (COUNT(?src) AS ?postings) (SUM(?n) as ?total) {
SELECT ?rel ?src (COUNT(?src) AS ?n)
WHERE {
    GRAPH ?src
    {[] a sdo:JobPosting; ?pred ?data }
    BIND (replace(str(?pred), 'https?://schema.org/(JobPosting/)?', '') AS ?rel)
}
GROUP BY ?rel ?src
}
GROUP BY ?rel
ORDER BY desc(?postings)
''')

CPU times: user 58.8 s, sys: 1.05 s, total: 59.8 s
Wall time: 1min 3s


In [47]:
df = pd.DataFrame([[value.toPython() for value in row] for row in results], columns = ['uri', 'n', 'total'])
df.assign(frac=lambda df: df.n/max(df.n)).assign(avg = lambda df: df.total / df.n).head(30)

,uri,n,total,frac,avg
0,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,13092,13092,1.000000,1.000000
1,title,11862,12052,0.906049,1.016018
2,description,11323,11540,0.864879,1.019165
3,datePosted,10420,10515,0.795906,1.009117
4,jobLocation,9800,10423,0.748549,1.063571
5,hiringOrganization,9568,9720,0.730828,1.015886
6,employmentType,7702,8139,0.588298,1.056739
7,validThrough,4688,4691,0.358081,1.000640
8,baseSalary,3657,3713,0.279331,1.015313
9,industry,3328,4081,0.254201,1.226262


### Titles are strings

In [38]:
%%time
results = dataset.query('''
PREFIX sdo: <http://schema.org/>
PREFIX sdo_jp: <http://schema.org/JobPosting>

SELECT ?type (COUNT(?src) AS ?n)
WHERE {
    GRAPH ?src
    {[] a sdo:JobPosting; sdo:title|sdo_jp:title ?data}
    BIND (datatype(?data) as ?type)
}
GROUP BY ?type
ORDER BY DESC(?n)
LIMIT 10
''')


head(results, 10)

['http://www.w3.org/2001/XMLSchema#string', 5296]
CPU times: user 2.14 s, sys: 78.1 ms, total: 2.22 s
Wall time: 2.4 s


### Descriptions are strings

In [39]:
%%time
results = dataset.query('''
PREFIX sdo: <http://schema.org/>
PREFIX sdo_jp: <http://schema.org/JobPosting>

SELECT ?type (COUNT(?src) AS ?n)
WHERE {
    GRAPH ?src
    {[] a sdo:JobPosting; sdo:description|sdo_jp:description ?data}
    BIND (datatype(?data) as ?type)
}
GROUP BY ?type
ORDER BY DESC(?n)
LIMIT 10
''')


head(results, 10)

['http://www.w3.org/2001/XMLSchema#string', 5288]
CPU times: user 2.12 s, sys: 78.1 ms, total: 2.2 s
Wall time: 2.34 s


baseSalary is normally a Monetary Amount

In [40]:
%%time
results = dataset.query('''
PREFIX sdo: <http://schema.org/>
PREFIX sdo_jp: <http://schema.org/JobPosting/>

SELECT ?type (COUNT(?src) as ?n)
WHERE {
    GRAPH ?src
    {[] a sdo:JobPosting; sdo:baseSalary|sdo_jp:baseSalary ?data .
     OPTIONAL {?data a ?datatype .}
     BIND (coalesce(datatype(?data), ?datatype) as ?type)}
}
GROUP BY ?type
ORDER BY DESC(?n)
LIMIT 20
''')


head(results, 20)

['http://schema.org/MonetaryAmount', 2946]
['http://www.w3.org/1999/02/22-rdf-syntax-ns#langString', 527]
['http://www.w3.org/2001/XMLSchema#string', 135]
['https://schema.org/MonetaryAmount', 72]
[None, 16]
['http://schema.org/PriceSpecification', 7]
['http:/schema.orgMonetaryAmount', 6]
['https://schema.org/PriceSpecification', 1]
['http://www.w3.org/2001/XMLSchema#boolean', 1]
['http://www.w3.org/2001/XMLSchema#integer', 1]
['http://schema.org/12000%20-%2015000t', 1]
CPU times: user 2.44 s, sys: 62.5 ms, total: 2.5 s
Wall time: 2.75 s


In [41]:
%%time
results = dataset.query('''
PREFIX sdo: <http://schema.org/>
PREFIX sdo_jp: <http://schema.org/JobPosting/>

SELECT ?rel (COUNT(?src) AS ?postings) (SUM(?n) as ?total) {
SELECT ?rel ?src (COUNT(?src) AS ?n)
WHERE {
    GRAPH ?src
    {[] a sdo:JobPosting; sdo:baseSalary|sdo_jp:baseSalary ?salary .
     ?salary ?pred ?data .}
    BIND (replace(str(?pred), 'https?://schema.org/(MonetaryAmount/)?', '') AS ?rel)
}
GROUP BY ?rel ?src
}
GROUP BY ?rel
ORDER BY desc(?postings)
''')

CPU times: user 5.97 s, sys: 156 ms, total: 6.12 s
Wall time: 6.38 s


In [42]:
df = pd.DataFrame([[value.toPython() for value in row] for row in results], columns = ['uri', 'n', 'total'])
df.assign(prop=lambda df: df.n/13093,
          frac=lambda df: df.n/max(df.n),
          avg = lambda df: df.total / df.n).head(30)

,uri,n,total,prop,frac,avg
0,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,3021,3033,0.230734,1.000000,1.003972
1,value,2826,2841,0.215841,0.935452,1.005308
2,currency,2706,2717,0.206675,0.895730,1.004065
3,minValue,143,143,0.010922,0.047335,1.000000
4,maxValue,141,141,0.010769,0.046673,1.000000
5,unitText,39,39,0.002979,0.012910,1.000000
6,PriceSpecification/maxPrice,6,6,0.000458,0.001986,1.000000
7,PriceSpecification/minPrice,6,6,0.000458,0.001986,1.000000
8,PriceSpecification/priceCurrency,6,6,0.000458,0.001986,1.000000
9,name,5,5,0.000382,0.001655,1.000000


In [43]:
%%time
results = dataset.query('''
PREFIX sdo: <http://schema.org/>
PREFIX sdo_jp: <http://schema.org/JobPosting/>
PREFIX sdo_ma: <http://schema.org/MonetaryAmount/>


SELECT ?type (COUNT(?src) as ?n)
WHERE {
    GRAPH ?src
    {[] a sdo:JobPosting; (sdo:baseSalary|sdo_jp:baseSalary)/(sdo:value|sdo_ma:value) ?data .
     OPTIONAL {?data a ?datatype .}
     BIND (coalesce(datatype(?data), ?datatype) as ?type)}
}
GROUP BY ?type
ORDER BY DESC(?n)
LIMIT 20
''')


head(results, 20)

['http://schema.org/QuantitativeValue', 2323]
['http://www.w3.org/1999/02/22-rdf-syntax-ns#langString', 280]
['http://www.w3.org/2001/XMLSchema#string', 152]
[None, 7]
['http://schema.org/PropertyValue', 4]
['https://schema.org/QuantitativeValue', 2]
['http://www.w3.org/2001/XMLSchema#boolean', 1]
['http:/schema.orgQuantitativeValue', 1]
['http://www.w3.org/2001/XMLSchema#integer', 1]
['http://www.w3.org/2001/XMLSchema#double', 1]
['http://schema.org/MonetaryAmount', 1]
CPU times: user 2.19 s, sys: 62.5 ms, total: 2.25 s
Wall time: 2.37 s
